In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import pandas as pd
import math

plt.rcParams["figure.figsize"] = (18, 9)

In [3]:
train_path = "andan-2022-autumn-lab-3/train.csv"
test_path = "andan-2022-autumn-lab-3/test_x.csv"

In [4]:
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

In [51]:
train_rubrics_set = set()
for i in train_data['rubrics']:
    train_rubrics_set.add(i)

def rubrics_handler(s):
    return [0.5 if s == i else 0 for i in train_rubrics_set]

columns = train_data.columns
name_to_id = dict()
org_id = None
rt = None
rubrics = None
def line_handler(line):
    # global columns
    # global name_to_id
    ma = []
    for name in columns:
        # print(name, ma)
        if name in ['rating_org',
                    'user_id',
                    'average_bill',
                    'ts',
                    'org_id',
                    ]: # ignore
            continue

        name_to_id[name] = len(ma)
        if False: pass
        elif name in ['rating']:
            ma += [int(line[name])]
            # ma += [int(line[name]) * 0.1]
        elif name in ['rubrics']:
            ma += rubrics_handler(line[name])
        elif name in ['org_city', 'user_city']:
            ma += [0 if line[name] == 'msk' else 1]
        else:
            ma += [line[name]]
    global org_id, rubrics, rt
    # org_id = name_to_id['org_id']
    rubrics = name_to_id['rubrics']
    rt = name_to_id['rating']
    return ma

def getRegression(data):
    X = []
    # columns = train_data.columns
    for i in data.index:
        line = data.iloc[i]
        X += [line_handler(line)]
        # print(X)

    print(X[0])
    X = np.array(X)
    print(X[0])
    # neigh = KNeighborsRegressor(n_neighbors=5, metric=metric, n_jobs=-1)
    # neigh.fit(X, rating_org)
    # return neigh
    return X


In [56]:
train_data_original = pd.read_csv(train_path)
train_data = train_data_original.copy()

In [57]:
X, rating_org = getRegression(train_data), train_data['rating_org']


[0.4, 0, 0, 0, 0, 0, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0.4 0.  0.  0.  0.  0.  0.  0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.
 1.  1.  0.  1.  1.  0.  0.  1.  1.  0.  1.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0. ]


In [58]:
X_with_org_original = list(zip(X, list(rating_org)))

In [59]:
errors = []
def pred(X, rating_org, test, rating_pred, neigh):
    neigh.fit(X, rating_org)
    from sklearn.metrics import mean_absolute_percentage_error
    prc = neigh.predict(test)
    errors.append(mean_absolute_percentage_error(rating_pred, prc))
    print(errors[-1])

for i in range(10):
    import random
    X_with_org = X_with_org_original.copy()
    random.shuffle(X_with_org)
    train_comb = X_with_org[int(len(X_with_org)*0.8):]
    test_comb = X_with_org[:int(len(X_with_org)*0.8)]
    X, rating_org = zip(*train_comb)
    test, rating_pred = zip(*test_comb)

    from sklearn.neighbors import KNeighborsRegressor
    neigh = KNeighborsRegressor(n_neighbors=int(len(train_comb)**0.5),
                                # weights='distance',
                                p=1,
                                n_jobs=-1,
                                )

    pred(X, rating_org, test, rating_pred, neigh)

print("Mean:", sum(errors)/len(errors))
# 1) n_neighbors=int(len(train_comb)**0.5), weights='uniform', p=2
# 0.04532506380033315
###
# 2) n_neighbors=int(len(train_data)**0.5), weights='uniform', p=1
# 0.04486101819674648
###
# 3) n_neighbors=int(len(train_data)**0.5), weights='distance', p=2
# 0.037119345809190775
###
# 4) n_neighbors=int(len(train_data)**0.5), weights='distance', p=1
# 0.03673000806608671
### (4) 6.96103 versus 6.17398 (2)

###
# 5) n_neighbors=int(len(train_data)**0.5), weights='uniform', p=1, rating*0.1
# 0.04501220177438429

0.04491370608322636
0.04502860305588225
0.044996010509156525
0.04487724987616019
0.04497818068732718
0.04509775973618921
0.04513572643030087
0.04479199945161292
0.04534759328646655
0.04495518862752081
Mean: 0.04501220177438429


Предсказание test_x.scv:

In [61]:
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

In [62]:
X, rating_org = getRegression(train_data), train_data['rating_org']

[0.4, 0, 0, 0, 0, 0, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0.4 0.  0.  0.  0.  0.  0.  0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.
 1.  1.  0.  1.  1.  0.  0.  1.  1.  0.  1.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0. ]


In [63]:
from sklearn.neighbors import KNeighborsRegressor
neigh = KNeighborsRegressor(n_neighbors=int(len(train_data)**0.5),
                            # weights='distance',
                            p=1,
                            n_jobs=-1,
                            )
neigh.fit(X, rating_org)

KNeighborsRegressor(n_jobs=-1, n_neighbors=272, p=1)

In [64]:
ma = neigh.predict(getRegression(test_data))

[0.5, 1, 1, 0, 0, 0, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0.5 1.  1.  0.  0.  0.  0.  0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.
 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0. ]


In [65]:
from itertools import count
rating_pred = pd.DataFrame(zip(count(0), np.array(ma)), columns=['id', 'rating_org'])
rating_pred.to_csv('andan-2022-autumn-lab-3/saved_ratings.csv', index=False)
print(rating_pred)

          id  rating_org
0          0    4.348607
1          1    4.368170
2          2    4.098761
3          3    4.322344
4          4    4.568826
...      ...         ...
24660  24660    4.131525
24661  24661    4.497799
24662  24662    4.361395
24663  24663    4.462154
24664  24664    4.427409

[24665 rows x 2 columns]
